<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Michael", "transactions": [{"transaction-id": 25, "amount": 774}, {"transaction-id": 61, "amount": 735}, {"transaction-id": 339, "amount": 951}, {"transaction-id": 360, "amount": 829}, {"transaction-id": 416, "amount": 910}, {"transaction-id": 490, "amount": 902}, {"transaction-id": 495, "amount": 722}, {"transaction-id": 614, "amount": 729}, {"transaction-id": 670, "amount": 805}, {"transaction-id": 859, "amount": 854}, {"transaction-id": 1126, "amount": 837}, {"transaction-id": 1240, "amount": 1079}, {"transaction-id": 1449, "amount": 784}, {"transaction-id": 1487, "amount": 800}, {"transaction-id": 1540, "amount": 863}, {"transaction-id": 1606, "amount": 861}, {"transaction-id": 1609, "amount": 891}, {"transaction-id": 1763, "amount": 834}, {"transaction-id": 1792, "amount": 867}, {"transaction-id": 1870, "amount": 621}, {"transaction-id": 1915, "amount": 743}, {"transaction-id": 1966, "amount": 742}, {"transaction-id": 2036, "amount": 887}, {"transaction-id": 2

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Michael", "transactions": [{"transaction-id": 25, "amount": 774}, {"transaction-id": 61, "amount": 735}, {"transaction-id": 339, "amount": 951}, {"transaction-id": 360, "amount": 829}, {"transaction-id": 416, "amount": 910}, {"transaction-id": 490, "amount": 902}, {"transaction-id": 495, "amount": 722}, {"transaction-id": 614, "amount": 729}, {"transaction-id": 670, "amount": 805}, {"transaction-id": 859, "amount": 854}, {"transaction-id": 1126, "amount": 837}, {"transaction-id": 1240, "amount": 1079}, {"transaction-id": 1449, "amount": 784}, {"transaction-id": 1487, "amount": 800}, {"transaction-id": 1540, "amount": 863}, {"transaction-id": 1606, "amount": 861}, {"transaction-id": 1609, "amount": 891}, {"transaction-id": 1763, "amount": 834}, {"transaction-id": 1792, "amount": 867}, {"transaction-id": 1870, "amount": 621}, {"transaction-id": 1915, "amount": 743}, {"transaction-id": 1966, "amount": 742}, {"transaction-id": 2036, "amount": 887}, {"transaction-id": 2

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Michael',
  'transactions': [{'transaction-id': 25, 'amount': 774},
   {'transaction-id': 61, 'amount': 735},
   {'transaction-id': 339, 'amount': 951},
   {'transaction-id': 360, 'amount': 829},
   {'transaction-id': 416, 'amount': 910},
   {'transaction-id': 490, 'amount': 902},
   {'transaction-id': 495, 'amount': 722},
   {'transaction-id': 614, 'amount': 729},
   {'transaction-id': 670, 'amount': 805},
   {'transaction-id': 859, 'amount': 854},
   {'transaction-id': 1126, 'amount': 837},
   {'transaction-id': 1240, 'amount': 1079},
   {'transaction-id': 1449, 'amount': 784},
   {'transaction-id': 1487, 'amount': 800},
   {'transaction-id': 1540, 'amount': 863},
   {'transaction-id': 1606, 'amount': 861},
   {'transaction-id': 1609, 'amount': 891},
   {'transaction-id': 1763, 'amount': 834},
   {'transaction-id': 1792, 'amount': 867},
   {'transaction-id': 1870, 'amount': 621},
   {'transaction-id': 1915, 'amount': 743},
   {'transaction-id': 1966, 'amount': 7

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 3,
  'name': 'Alice',
  'transactions': [{'transaction-id': 634, 'amount': -107},
   {'transaction-id': 3450, 'amount': -26},
   {'transaction-id': 4742, 'amount': -68},
   {'transaction-id': 8842, 'amount': -36}]},
 {'id': 10,
  'name': 'Alice',
  'transactions': [{'transaction-id': 62, 'amount': 1095},
   {'transaction-id': 110, 'amount': 666},
   {'transaction-id': 163, 'amount': 399},
   {'transaction-id': 583, 'amount': 785},
   {'transaction-id': 831, 'amount': 495},
   {'transaction-id': 952, 'amount': 839},
   {'transaction-id': 1246, 'amount': 806},
   {'transaction-id': 1378, 'amount': 592},
   {'transaction-id': 1546, 'amount': 77},
   {'transaction-id': 1765, 'amount': 512},
   {'transaction-id': 2214, 'amount': 504},
   {'transaction-id': 2327, 'amount': 220},
   {'transaction-id': 2476, 'amount': 407},
   {'transaction-id': 3219, 'amount': 1190},
   {'transaction-id': 3426, 'amount': 58},
   {'transaction-id': 3719, 'amount': 419},
   {'transaction-id': 3859, 'amo

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 4},
 {'name': 'Alice', 'count': 51},
 {'name': 'Alice', 'count': 11},
 {'name': 'Alice', 'count': 60},
 {'name': 'Alice', 'count': 69})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(4, 51, 11, 60, 69)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

38.62880886426593

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 634, 'amount': -107},
  {'transaction-id': 3450, 'amount': -26},
  {'transaction-id': 4742, 'amount': -68},
  {'transaction-id': 8842, 'amount': -36}],
 [{'transaction-id': 62, 'amount': 1095},
  {'transaction-id': 110, 'amount': 666},
  {'transaction-id': 163, 'amount': 399},
  {'transaction-id': 583, 'amount': 785},
  {'transaction-id': 831, 'amount': 495},
  {'transaction-id': 952, 'amount': 839},
  {'transaction-id': 1246, 'amount': 806},
  {'transaction-id': 1378, 'amount': 592},
  {'transaction-id': 1546, 'amount': 77},
  {'transaction-id': 1765, 'amount': 512},
  {'transaction-id': 2214, 'amount': 504},
  {'transaction-id': 2327, 'amount': 220},
  {'transaction-id': 2476, 'amount': 407},
  {'transaction-id': 3219, 'amount': 1190},
  {'transaction-id': 3426, 'amount': 58},
  {'transaction-id': 3719, 'amount': 419},
  {'transaction-id': 3859, 'amount': 504},
  {'transaction-id': 3896, 'amount': -6},
  {'transaction-id': 4096, 'amount': 676},
  {'transaction-id

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 634, 'amount': -107},
 {'transaction-id': 3450, 'amount': -26},
 {'transaction-id': 4742, 'amount': -68})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(-107, -26, -68)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

863.2411617067049

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 173), ('Alice', 173), ('Alice', 187), ('Alice', 189), ('Bob', 95), ('Bob', 95), ('Bob', 99), ('Bob', 102), ('Charlie', 109), ('Charlie', 111), ('Charlie', 117), ('Charlie', 118), ('Dan', 178), ('Dan', 179), ('Dan', 195), ('Dan', 195), ('Edith', 140), ('Edith', 143), ('Edith', 152), ('Edith', 154), ('Frank', 142), ('Frank', 146), ('Frank', 154), ('Frank', 158), ('George', 132), ('George', 132), ('George', 142), ('George', 143), ('Hannah', 95), ('Hannah', 96), ('Hannah', 104), ('Hannah', 104), ('Ingrid', 120), ('Ingrid', 120), ('Ingrid', 130), ('Ingrid', 130), ('Jerry', 36), ('Jerry', 36), ('Jerry', 39), ('Jerry', 39), ('Kevin', 130), ('Kevin', 130), ('Kevin', 142), ('Kevin', 143), ('Laura', 65), ('Laura', 65), ('Laura', 120), ('Michael', 180), ('Michael', 180), ('Michael', 195), ('Michael', 195), ('Norbert', 53), ('Norbert', 56), ('Norbert', 59), ('Norbert', 59), ('Oliver', 132), ('Oliver', 132), ('Oliver', 143), ('Oliver', 143), ('Patricia', 204), ('Patricia', 204), ('Patric

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 722), ('Bob', 391), ('Charlie', 455), ('Dan', 747), ('Edith', 589), ('Frank', 600), ('George', 549), ('Hannah', 399), ('Ingrid', 500), ('Jerry', 150), ('Kevin', 545), ('Laura', 250), ('Michael', 750), ('Norbert', 227), ('Oliver', 550), ('Patricia', 850), ('Quinn', 400), ('Ray', 591), ('Sarah', 546), ('Tim', 132), ('Ursula', 150), ('Victor', 470), ('Wendy', 428), ('Xavier', 500), ('Yvonne', 350), ('Zelda', 396)]
CPU times: user 83.7 ms, sys: 7.97 ms, total: 91.7 ms
Wall time: 335 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Michael,"[{'transaction-id': 25, 'amount': 774}, {'tran..."
1,1,Quinn,"[{'transaction-id': 1658, 'amount': 201}, {'tr..."
2,2,Dan,"[{'transaction-id': 839, 'amount': -199}, {'tr..."
3,3,Alice,"[{'transaction-id': 634, 'amount': -107}, {'tr..."
4,4,Tim,"[{'transaction-id': 15, 'amount': 409}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 105 ms, sys: 10 ms, total: 115 ms
Wall time: 1.02 s


name
Alice      722
Bob        391
Charlie    455
Dan        747
Edith      589
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Michael', 'amount': 774, 'transaction-id': 25},
 {'id': 0, 'name': 'Michael', 'amount': 735, 'transaction-id': 61},
 {'id': 0, 'name': 'Michael', 'amount': 951, 'transaction-id': 339})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Michael,774,25
1,0,Michael,735,61
2,0,Michael,951,339
3,0,Michael,829,360
4,0,Michael,910,416


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 101 ms, sys: 3.99 ms, total: 105 ms
Wall time: 743 ms


name
Alice       27890
Bob         17437
Charlie     14810
Dan         32724
Edith       24902
Frank       26707
George      24625
Hannah      16628
Ingrid       7586
Jerry        6640
Kevin       17507
Laura       22717
Michael     32681
Norbert      4631
Oliver      15254
Patricia    42384
Quinn       18159
Ray         15845
Sarah       19215
Tim          4551
Ursula      12388
Victor      25496
Wendy       15842
Xavier      23873
Yvonne      21305
Zelda        8203
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()